In [49]:
# -*- coding: utf-8 -*
import twitter
import csv
import json
import re
import requests
import lxml.html as html
from urllib2 import urlopen
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz
import time


In [60]:
class TJLoader:

    def __init__(self):
        self._news_pages = ["https://tjournal.ru/paper/page/{}", "https://tjournal.ru/club/news/recent/page/{}"]
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        for news_page in self._news_pages:
            
            for i in range(count):
                page = html.parse(urlopen(news_page.format(i+min_index)))
                divs = page.getroot().find_class('b-articles__b__title')

                for div in divs:
                    links.append(div.getchildren()[1].get("href"))

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с tjournal, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__title")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("b-article__infoline__comments")
        comment = int(comments[0].find("b").text.replace(" ", ""))

        # Теги
        tags = root.find_class("b-article__tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infoline__date")
        date = self._parse_date(date[0].text)
        
        # Первоисточник
        source = root.find_class("b-article__link")
        if len(source) != 0:
            source = source[0].getchildren()[0].getchildren()[1].getchildren()[0].text
        else:
            source = None
        
        # Type
        if "tjournal.ru/c/" in link:
            news_type = "TJ_C"
        else:
            news_type = "TJ_P"

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date, "url": link, "type": news_type, "source": source}


    def get_tj_news_info(self, min_index=1, count=30, first_date="2010-01-01", last_date="2017-01-01"):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :last_news: str, id последней новости, которую нужно скачать
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        is_break = False
        for link in links:
            
            link_info = self.get_link_info(link)
            # Если заданное время не подходит
            if link_info["date"] > first_date or link_info["date"] < last_date:
                continue
    
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [61]:
class VCLoader:

    def __init__(self):
        self._news_pages = "https://api.vc.ru/1/paper"
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        text = requests.get(self._news_pages).text
        json_req = json.loads(text)
    
        for news in json_req:
            links.append(news["url"])

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с vc, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__head")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("ccount")[0].text
        comment = int(comments.replace(" ", ""))

        # Теги
        tags = root.find_class("b-tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infopanel__date")
        date = self._parse_date(date[0].text)
        

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date, "url": link, "type": "VC"}


    def get_tj_news_info(self, min_index=1, count=30, first_date="2010-01-01", last_date="2017-01-01"):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :first_date: str, дата и время первой (самой новой) новости
        :last_date: str, дата и время последней(самой старой) новости
        
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        
        for link in links:
            
            link_info = self.get_link_info(link)
            # Если заданное время не подходит
            if link_info["date"] > first_date or link_info["date"] < last_date:
                continue
            
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [62]:
class TwitterLoader:


    def __init__(self):
        CONSUMER_KEY = 'BOuuaMDhNhm6yx0rzqK8bMsbI'
        CONSUMER_SECRET = '3DybJwlkXd2vU6R385yLA8yJblYJltLtwojySD9AVs04ShauZ0'

        ACCESS_TOKEN_KEY = '3712177576-of3jzZ8gNmlPDfPjPyR0Ljw1Ao2IXdTqX9dZGDZ'
        ACCESS_TOKEN_SECRET = 'Ky7iKwByHNXX3UMfuMhv6UgVx2IhjLo3KmwpsBQz35wtG'

        self.api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"



    def _parse_date(self, date):

        """

        :param date: str, дата в формате твиттера - "Sat Nov 21 17:00:29 +0000 2015"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date_array = date.split(' ')
        month = self._month_dict[date_array[1]]
        day = int(date_array[2])
        time = date_array[3]
        year = date_array[5]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        return str(moscow_date).split("+")[0]



    def loadTweetWithLink(self, link, date):
        """

        :param link: str, ключевое слово для поиска, вданном случае - ссылка на письмо
        :param date: str, дата, до которой искать твиты
        :return: список словариков с информацией о УНИКАЛЬНЫХ твитах по запросу link
        """
        set_id = set()
        tweet_list = []
        result = self.api.GetSearch(term=link, until=date, count=100000)

        for res in result:

            tw_id = res.GetId()
            # Если мы уже обрабатывали этот твит - идем дальше
            if tw_id in set_id:
                continue

            retweeted_status = res.GetRetweeted_status()
            # Если это не ретвит
            if retweeted_status is None:
                is_retweet = 0
                retweeted_count = res.GetRetweetCount()
                favorite_count = res.GetFavoriteCount()
            else:
                is_retweet = 1
                retweeted_count = 0
                favorite_count = 0
            set_id.add(tw_id)

            created_at = res.GetCreatedAt()
            created_at = self._parse_date(created_at)

            # Данные о пользователе
            user = res.GetUser()
            followers_count = user.followers_count
            listed_count = user.listed_count
            friends_count = user.friends_count
            favourites_count = user.favourites_count
            statuses_count = user.statuses_count


            tw_dict= {
                    "url": link,
                    "tw_id":tw_id,
                    "retweeted_count": retweeted_count,
                    "favorite_count":favorite_count,
                    "is_retweet": is_retweet,
                    "created_at":created_at,
                    "user_followers_count":followers_count,
                    "user_listed_count": listed_count,
                    "user_friends_count":friends_count,
                    "user_favourites_count":favourites_count,
                    "user_statuses_count":statuses_count
                    }

            tweet_list.append(tw_dict)

        return tweet_list

    def _get_next_date(self, date, days):
        """

        :param date: str, дата
        :param days: int, количество дней
        :return: возвращает дату через days-дней после date
        """
        date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        date += timedelta(days=days)
        return str(date).split(' ')[0]


    def load_tweets_by_term(self, news_list, days_after_news=2):
        """

        :param news_list: list(str), список словариков с информацией о новости
        :param days_after_news:  int, количество дней после публикации новости, до которой искать
        """
        result_list = []
        i = 0
        for news in news_list:
            tweets = []
            try:
                until_date = self._get_next_date(news["date"], days_after_news)
                tweets = self.loadTweetWithLink(news["url"], until_date)
            except twitter.error.TwitterError as ex:
                print str(ex)
                if str(ex) == self._RATE_LIMIT:
                    sleep_time = self.api.GetSleepTime("search/tweets") #??? Почему-то не работает
                    print "Спим {} сек.".format(sleep_time)
                    time.sleep(sleep_time+2)
                    tweets = self.loadTweetWithLink(news["url"], until_date)

            result_list+= tweets
            
            i+=1
            if i%10 == 0:
                print "Собрано информация о", i, " новостях"
            
        return result_list

In [64]:
loader = TJLoader()
tj_pages = loader.get_tj_news_info(min_index=0, count=4, first_date="2015-11-25 14:00", last_date="2015-11-17 10:00")

Скачали  10  страниц
Скачали  20  страниц
Скачали  30  страниц
Скачали  40  страниц
Скачали  50  страниц
Скачали  60  страниц
Скачали  70  страниц
Скачали  80  страниц
Скачали  90  страниц
Скачали  100  страниц
Скачали  110  страниц
Скачали  120  страниц
Скачали  130  страниц
Скачали  140  страниц
Скачали  150  страниц
Скачали  160  страниц
Скачали  170  страниц
Скачали  180  страниц


In [72]:
vc = VCLoader()
vc_pages = vc.get_tj_news_info(first_date="2015-11-25 14:00", last_date="2015-11-17 10:00")

Скачали  10  страниц
Скачали  20  страниц
Скачали  30  страниц
Скачали  40  страниц


In [73]:
pages = vc_pages + tj_pages

In [74]:
OUT_NEWS_FILE = "news.csv"
OUT_TWITTER_FILE = "twitter.csv"

In [279]:
news_info = pd.DataFrame(pages)
news_info.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [280]:
tw = TwitterLoader()
tweeter_data = tw.load_tweets_by_term(pages)

KeyboardInterrupt: 

In [ ]:
tweeter_data = pd.DataFrame(tweeter_data)
tweeter_data.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [398]:
# !!!!   !!!!!!!!!! !!!!!!!!!!1 !!!!!!!!!!!!
# Сначала почищу данные. Там, где не материалы редакции - мусор, на который твитов то почти и нет!
news_info = news_info[ (news_info['type'] == "VC") | (news_info["type"] == "TJ_P")]
len(news_info["url"].unique())

112

In [399]:
# Мерджим дата фреймы
df = news_info.merge(tweeter_data, on='url', left_index=False, right_index=False)

In [400]:
df.head(4)

,comments,date,source,tags,title,type,url,views,created_at,favorite_count,is_retweet,retweeted_count,tw_id,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,15,2015-11-25 12:56,NaN,"[роботы, тест, исследование, рекрутинг]",Роботы решают всё,VC,https://vc.ru/p/hr-robots,729,2015-11-25 13:43:43,0,1,0,669466486570917889,1137,342,148,26,39371
1,15,2015-11-25 12:56,NaN,"[роботы, тест, исследование, рекрутинг]",Роботы решают всё,VC,https://vc.ru/p/hr-robots,729,2015-11-25 13:42:04,4,0,1,669466071150305280,9,57750,6,666,15281
2,18,2015-11-25 11:57,NaN,[],Gett позволит вызвать мастера для замены экран...,VC,https://vc.ru/n/gett-iphone,2105,2015-11-25 15:01:53,0,1,0,669486160104824832,95,13,28,0,3903
3,18,2015-11-25 11:57,NaN,[],Gett позволит вызвать мастера для замены экран...,VC,https://vc.ru/n/gett-iphone,2105,2015-11-25 13:52:09,0,0,0,669468608909713408,4997,576,58,46,21629


In [401]:
def diff_date_minutes(news_date, tweet_date):
    news_date = datetime.strptime(news_date, '%Y-%m-%d %H:%M')
    tweet_date = datetime.strptime(tweet_date, '%Y-%m-%d %H:%M:%S')
    return int((tweet_date-news_date).total_seconds()/60)

In [402]:
# количество минут, с момента публикации записи, данного твита
df["time_since_news"] = df.apply(lambda s: diff_date_minutes(s["date"], s["created_at"]), axis=1)

In [403]:
def get_week_day(date):
    date = datetime.strptime(date, '%Y-%m-%d %H:%M')
    return date.weekday()

In [404]:
# индекс дня недели
df["week_day_news"] = df.date.apply(lambda s: get_week_day(s))

In [405]:
def get_minutes_since_midnight(date):
    midnight = date.split(" ")[0] + " 00:00"
    date = datetime.strptime(date, '%Y-%m-%d %H:%M')
    midnight = datetime.strptime(midnight, '%Y-%m-%d %H:%M')
    return int((date-midnight).total_seconds()/60)

In [406]:
# Сколько минут времени прошло с полуночи
df["minutes_since_midnight"] = df.date.apply(lambda s: get_minutes_since_midnight(s))

### Агрегируем данные из твиттера

In [407]:
# Время для вычисления независимых переменных
FIRST_TIME = 10
# Время для вычисления целевой функции
LAST_TIME = 180

In [408]:
len(df["url"].unique())

112

In [409]:
ft_df = df[df["time_since_news"] <= FIRST_TIME]

In [410]:
grouped = ft_df.groupby("url")

In [411]:
# Считаем общее количество твиттов
count_of_tweets = pd.DataFrame(grouped["url"].count())
count_of_tweets.columns = ["first_time_tweet"]
count_of_tweets.reset_index(inplace=True)  
df = pd.merge(df, count_of_tweets, on='url', left_index=True, right_index=False, how="outer")

In [412]:
# Считаем общую аудиторию
follower_sum = pd.DataFrame(grouped["user_followers_count"].sum())
follower_sum.columns = ["follower_sum"]
follower_sum.reset_index(inplace=True)  
df = pd.merge(df, follower_sum, on='url', left_index=True, right_index=False, how="outer")

In [413]:
# Считаем число ретвитов
retweeted_count_sum = pd.DataFrame(grouped["retweeted_count"].sum())
retweeted_count_sum.columns = ["retweeted_count_sum"]
retweeted_count_sum.reset_index(inplace=True)  
df = pd.merge(df, retweeted_count_sum, on='url', left_index=True, right_index=False, how="outer")

In [414]:
# Считаем число звездочек
favorite_count_sum = pd.DataFrame(grouped["favorite_count"].sum())
favorite_count_sum.columns = ["favorite_count_sum"]
favorite_count_sum.reset_index(inplace=True)  
df = pd.merge(df, favorite_count_sum, on='url', left_index=True, right_index=False, how="outer")

In [415]:
# Считаем общее число списков, в которых состоят сделавшие посты
user_listed_count = pd.DataFrame(grouped["user_listed_count"].sum())
user_listed_count.columns = ["favorite_count"]
user_listed_count.reset_index(inplace=True)  
df = pd.merge(df, user_listed_count, on='url', left_index=True, right_index=False, how="outer")

In [416]:
st_df = df[df["time_since_news"] <= LAST_TIME]

In [417]:
# Выделяем значение целевой функции
grouped = st_df.groupby("url")
count_of_tweets = pd.DataFrame(grouped["url"].count())
count_of_tweets.columns = ["last_time_tweet"]
count_of_tweets.reset_index(inplace=True)  
df = pd.merge(df, count_of_tweets, on='url', left_index=True, right_index=False, how="outer")

In [418]:
# Сохраняемся
df.to_csv("prepared_to_analys.csv", sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [419]:
len(df["url"].unique())

112

### Аналитим 

In [422]:
len(df[["first_time_tweet","last_time_tweet"]].drop_duplicates().dropna())

58

In [395]:
df.head()

,comments,date,source,tags,title,type,url,views,created_at,favorite_count_x,...,user_statuses_count,time_since_news,week_day_news,minutes_since_midnight,first_time_tweet,follower_sum,retweeted_count_sum,favorite_count_sum,favorite_count_y,last_time_tweet
69,15,2015-11-25 12:56,NaN,"[роботы, тест, исследование, рекрутинг]",Роботы решают всё,VC,https://vc.ru/p/hr-robots,729,2015-11-25 13:43:43,0,...,39371,47,2,776,NaN,NaN,NaN,NaN,NaN,NaN
69,15,2015-11-25 12:56,NaN,"[роботы, тест, исследование, рекрутинг]",Роботы решают всё,VC,https://vc.ru/p/hr-robots,729,2015-11-25 13:42:04,4,...,15281,46,2,776,NaN,NaN,NaN,NaN,NaN,NaN
62,18,2015-11-25 11:57,NaN,[],Gett позволит вызвать мастера для замены экран...,VC,https://vc.ru/n/gett-iphone,2105,2015-11-25 15:01:53,0,...,3903,184,2,717,2,57799,2,2,669,2
62,18,2015-11-25 11:57,NaN,[],Gett позволит вызвать мастера для замены экран...,VC,https://vc.ru/n/gett-iphone,2105,2015-11-25 13:52:09,0,...,21629,115,2,717,2,57799,2,2,669,2
62,18,2015-11-25 11:57,NaN,[],Gett позволит вызвать мастера для замены экран...,VC,https://vc.ru/n/gett-iphone,2105,2015-11-25 13:49:20,0,...,714,112,2,717,2,57799,2,2,669,2


In [ ]:
# Удаляем лишние столбцы
major_features = ["url", ]

# После первого запуска, через несколько дней, запускать только часть, которая находится под этим заголовком!

In [222]:
# читаем данные
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
twitter_df = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [227]:
# находим самую "молодую новость"
news_df = news_df.sort_values(by=["date"], ascending=False)
last_news = news_df[news_df.index==0]["url"][0]

In [228]:
# скачивае информацию о новостях
loader = TJLoader()
pages = loader.get_tj_news_info(min_index=1, count=6, last_news=last_news)

In [229]:
len(pages)

0

In [ ]:
# Сохраняем в файлик
news_df = news_df.append(pd.DataFrame(pages))
news_df.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Скачиваем данные из твиттера
tw = TwitterLoader()
tweeter_data = tw.load_tweets_by_term(pages)

In [ ]:
len(tweeter_data)

In [ ]:
# Сохраняем в файлик
twitter_df = twitter_df.append(pd.DataFrame(tweeter_data))
twitter_df.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [5]:
import json

In [7]:
text = requests.get("https://api.vc.ru/1/paper").text

In [8]:
json_req = json.loads(text)

'\xd0\x9f\xd0\x9e\xd0\xa6\xd0\x95\xd0\x9b\xd0\xa3\xd0\x99'

In [19]:
page = html.parse(urlopen("https://vc.ru/p/interview-it"))
root = page.getroot()
print root

<Element html at 0x7f8cf07d6628>


In [26]:
page = html.parse(urlopen("https://vc.ru/n/microsoft-store-russia"))
root = page.getroot()

# заголовок
title = root.find_class("b-article__head")
title = title[0].find("h1").text
print title

# парсим количество просмотров
view = root.get_element_by_id("hitsCount").text
view = view.replace(" ", "")
view = int(view)
print view

# Количество комментариев
comments = root.find_class("ccount")[0].text
comment = int(comments.replace(" ", ""))
print comment

tags = root.find_class("b-tags__tag")
tag_list = []
for tag in tags:
    tag_list.append(tag.text)
    
print tag_list

Microsoft открыла собственный онлайн-магазин в России
5865
19
['microsoft', u'\u0438\u043d\u0442\u0435\u0440\u043d\u0435\u0442-\u043c\u0430\u0433\u0430\u0437\u0438\u043d']


In [22]:
print title

Какие вы любите сэндвичи


In [32]:
vc = VCLoader()

In [33]:
vc.get_link_info("https://vc.ru/n/microsoft-store-russia")

{'comments': 19,
 'date': u'2015-11-24 22:13',
 'tags': ['microsoft',
  u'\u0438\u043d\u0442\u0435\u0440\u043d\u0435\u0442-\u043c\u0430\u0433\u0430\u0437\u0438\u043d'],
 'title': u'Microsoft \u043e\u0442\u043a\u0440\u044b\u043b\u0430 \u0441\u043e\u0431\u0441\u0442\u0432\u0435\u043d\u043d\u044b\u0439 \u043e\u043d\u043b\u0430\u0439\u043d-\u043c\u0430\u0433\u0430\u0437\u0438\u043d \u0432 \u0420\u043e\u0441\u0441\u0438\u0438',
 'views': 5915}

In [35]:
vc_dict = vc.get_tj_news_info()

Скачали  10  страниц
Скачали  20  страниц
Скачали  30  страниц
Скачали  40  страниц
Скачали  50  страниц


Вопросы:
1) Что делать с соурсе? Эти новости никто не репостит с TJournal, а от других источников их можно найти